In [36]:
import cv2
import numpy as np
from threading import Thread

# Función para obtener el centro de un cuadro delimitador
def get_center(box):
    x, y, w, h = box
    return (x + w // 2, y + h // 2)

# Función para calcular la intersección sobre la unión (IoU) de dos cuadros delimitadores
def iou(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1 + w1, x2 + w2)
    yi2 = min(y1 + h1, y2 + h2)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = w1 * h1
    box2_area = w2 * h2
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area

# Función para encontrar el tracker más cercano a una detección
def find_closest_tracker(detection, trackers, threshold=0.2):
    closest_tracker = None
    max_iou = 0
    for tracker in trackers:
        overlap = iou(detection, tracker.last_seen)
        if overlap > max_iou and overlap > threshold:
            closest_tracker = tracker
            max_iou = overlap
    return closest_tracker

# Clase para gestionar el seguimiento de una persona
class PersonTracker:
    def __init__(self, initial_detection, color, frame, id, life=10, max_inactive_frames=180):
        self.tracker = cv2.TrackerKCF_create()
        self.tracker.init(frame, tuple(initial_detection))
        self.color = color
        self.last_seen = initial_detection
        self.life = life
        self.inactive_frames = 0
        self.max_inactive_frames = max_inactive_frames
        self.static_frames = 0  # Contador de frames estáticos
        x, y, w, h = initial_detection
        self.template = cv2.cvtColor(frame[y:y+h, x:x+w], cv2.COLOR_BGR2GRAY)
        self.id = id
        self.last_center = get_center(initial_detection)

    def update(self, frame):
        success, box = self.tracker.update(frame)
        if success:
            self.last_seen = box
            self.life = 10
            self.inactive_frames = 0  # Restablecer el contador de inactividad al detectar con éxito
            current_center = get_center(box)
            if current_center == self.last_center:
                self.static_frames += 1
            else:
                self.static_frames = 0
            self.last_center = current_center
        else:
            self.life -= 1
            self.inactive_frames += 1  # Aumentar el contador de inactividad si no se detecta
        return success, box

# Función para realizar la comparación de plantillas
def match_template(frame, template):
    res = cv2.matchTemplate(frame, template, cv2.TM_CCOEFF_NORMED)
    _, max_val, _, max_loc = cv2.minMaxLoc(res)
    return max_val, max_loc

# Inicializar el clasificador Haar y el objeto de video
haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_fullbody.xml')
cap = cv2.VideoCapture("people_walking2.mp4")

# Variables para gestionar los trackers y colores
trackers = []
next_id = 0

def generate_unique_id():
    global next_id
    result = next_id
    next_id += 1
    return result

def generate_new_color():
    return tuple(np.random.randint(0, 255, size=(3)).tolist())

# Configuración para optimización
scale_factor = 0.5
detection_interval = 3  # Aumentar la frecuencia de detección
frame_count = 0
max_width = 150  # Reducir el tamaño máximo del área de detección
max_height = 150  # Reducir el tamaño máximo del área de detección
previous_detections = {}

# Función para procesar los frames en un hilo separado
def process_frame():
    global frame_count, trackers, previous_detections

    while cap.isOpened():
        retVal, frame = cap.read()
        if not retVal:
            break

        frame = cv2.resize(frame, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_AREA)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        frame_count += 1
        if frame_count % detection_interval == 0:
            bodies = haar_cascade.detectMultiScale(gray, 1.05, 8, minSize=(30, 30), maxSize=(max_width, max_height))
            used_boxes = []  # Mantener seguimiento de las detecciones ya asignadas

            for (x, y, w, h) in bodies:
                if w > max_width or h > max_height:
                    continue

                detection_box = (x, y, w, h)
                closest_tracker = find_closest_tracker(detection_box, trackers)

                if closest_tracker is None:
                    best_match_id = None
                    best_match_val = 0.15  # Reducir la sensibilidad de la coincidencia
                    for id, data in previous_detections.items():
                        template, _ = data
                        max_val, _ = match_template(gray, template)
                        if max_val > best_match_val:
                            best_match_val = max_val
                            best_match_id = id

                    if best_match_id is not None and best_match_id not in [tracker.id for tracker in trackers]:
                        color = previous_detections[best_match_id][1]
                    else:
                        color = generate_new_color()
                        best_match_id = generate_unique_id()

                    new_tracker = PersonTracker(detection_box, color, frame, best_match_id)
                    if not any(iou(detection_box, tracker.last_seen) > 0.3 for tracker in trackers):
                        trackers.append(new_tracker)
                        previous_detections[best_match_id] = (new_tracker.template, color)
                        used_boxes.append(detection_box)
                else:
                    # Actualizar el tracker existente con la nueva detección si no está asignado aún
                    if detection_box not in used_boxes:
                        used_boxes.append(detection_box)
                        closest_tracker.last_seen = detection_box
                        closest_tracker.inactive_frames = 0  # Restablecer el contador de inactividad

        for tracker in trackers:
            success, box = tracker.update(frame)
            if not success:
                max_val, max_loc = match_template(gray, tracker.template)
                if max_val > 0.75:  # Reducir la sensibilidad de la coincidencia de plantillas
                    x, y = max_loc
                    w, h = tracker.template.shape[1], tracker.template.shape[0]
                    tracker.last_seen = (x, y, w, h)
                    tracker.life = 3
                    tracker.inactive_frames = 0  # Restablecer el contador de inactividad
                    used_template_matching = True
                else:
                    used_template_matching = False
            else:
                used_template_matching = False

            if success or used_template_matching:
                (x, y, w, h) = [int(v) for v in tracker.last_seen]
                cv2.rectangle(frame, (x, y), (x+w, y+h), tracker.color, 2)
                if used_template_matching:
                    cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)

                text = f"Persona {tracker.id + 1}"
                text_position = (x, y + h + 20)
                cv2.putText(frame, text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # Eliminar trackers inactivos durante demasiado tiempo o si están estáticos por más de 6 segundos
        trackers = [tracker for tracker in trackers if tracker.life > 0 and tracker.inactive_frames < tracker.max_inactive_frames and tracker.static_frames < 180]

        cv2.imshow('Frame', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Crear un hilo para procesar los frames
thread = Thread(target=process_frame)
thread.start()
thread.join()
